In [1]:
import numpy as np
import pandas as pd
import math
from collections import Counter

In [2]:
class Node:
    
    # values after '*' must be passed as keyword
    # like value = 'Yes' if just 'Yes' will raise error 
    def __init__(self, feature = None, threshold = None, left = None, right = None, * , value = None):

        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    # Leaf node are those which don't split any further

    def is_leaf_node(self):
        return self.value is not None
        # True if the node is not None

In [3]:
X = np.array([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
    [4.7, 3.2, 1.3, 0.2]
])

print(X.shape)    # (3, 4)
print(X.shape[0]) # 3 → rows
print(X.shape[1]) 

(3, 4)
3
4


In [ ]:
class DecisionTree:

    # n_features are basically the no. of features

    def __init__(self, max_depth = 20, min_sample_split = 2, n_features = None):
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X , y, 0 )
    
    def _grow_tree(self, X, y, depth):
        
        # Agains samples are the number of row and feature gives the number of col
        n_samples, n_feat = X.shape

        # n_labels gets the unique classes from the y
        n_labels = len(np.unique(y))

        # Checking the stopping crietria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Find the best split
        feat_idx = np.random.choice(n_feat, self.n_features, replace=False)

        best_feature, best_thres = self._best_split(X, y, feat_idx)

        # Create the child node
        left_idx, right_idx = self._split(X[:, best_feature], best_thres)
        left = self._grow_tree(X[left_idx, :], y[left_idx], depth+1)
        right = self._grow_tree(X[right_idx, :], y[right_idx], depth+1)
        
        return Node(feature=best_feature, threshold=best_thres, left=left, right=right)


    def _most_common_label(self, y):
        count = Counter(y)
        value = count.most_common(1)[0][0]
        return value
    
    def _best_split(self, X, y, feat_idx):
        best_gain = -1
        split_idx, split_threshold = None, None

        for idx in feat_idx:
            X_col = X[:, idx]
            threshold = np.unique(X_col)

            for thr in threshold:

                # here we need to get the gain
                gain = self._information_gain(X_col, y , thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = idx
                    split_threshold = thr
        
        return split_idx, split_threshold

    def _information_gain(self, X_col, y, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        #create children
        left_idx, right_idx = self._split(X_col, threshold)

        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0

        #calculate the weight avg. entropy of children (subset)

        n = len(y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])

        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        #calculate  IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _entropy(self, y):

        # bincount gives a total count of each value [1,1,3] -> [0, 2, 1] its 0 has 0 count 1 has 2 count and 3 has 1 count

        hist = np.bincount(y)
        ps = hist / len(y)

        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _split(self, X_col, split_thres):
        left_idxs = np.argwhere(X_col <= split_thres).flatten()
        right_idxs = np.argwhere(X_col > split_thres).flatten()

        return left_idxs, right_idxs

    def predict(self, X):
        return np.array([self._traverse_tree(i, self.root) for i in X])
    
    def _traverse_tree(self, i, node):
        if node.is_leaf_node():
            return node.value
        
        if i[node.feature] <= node.threshold:
            return self._traverse_tree(i, node.left)
        
        return self._traverse_tree(i, node.right)

Entropy basically means disorder of the info
- Entropy formula:
- H(S)=−∑ pi* ​log2​(pi​) where pi is the probability of occurance of ith outcome

Information gain measures the entropy reduced after splitting the dataset

- IG = Entropy(before split) - Weighted average entropy(after split)

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size= 0.2
)

In [6]:
dt = DecisionTree(
    
)

dt.fit(X_train, y_train)

ydt_pred = dt.predict(X_test)

print(accuracy_score(y_test, ydt_pred))

0.9385964912280702


In [8]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()

tree.fit(X_train, y_train)

ydt_pred = tree.predict(X_test)

print(accuracy_score(y_test, ydt_pred))

0.9473684210526315
